### Tree of Thought
- Structures an LLM's reasoning process like a decision tree
- Allowing it to explore, evaluate, and backtrack through multiple reasoning paths (thoughts) to solve complex problems,

In [15]:
from langchain_core.tools import tool 


@tool 
def search_flights(origin:str , destination:str , date:str) -> dict:
    """Search for the cheapest flights"""
    return {'airline':'Indigo' , 'price':'4500' , 'arrival':'4.00' , 'departure':'6.00'}

@tool
def search_hotels(city:str , max_price: int)-> dict:
    """Search for the best hotel under budget"""
    return {'hotel':'Novotel' , "price_per_night": 3500,"rating": 4.2}

@tool
def  calculate_total_cost(flight_price :int , hotel_price: int , night: int )-> int:
    """Returns the totall cost"""
    return flight_price +  (hotel_price * night)

@tool
def format_flight_summary(hotel :dict , flight : dict ,  total_cost: int)-> str:
    """Formats flight plan summary"""
    return (
        f"FLIGHT: {flight['airline']}, {flight['price']},{flight['departure']}, {flight['arrival']}\n"
        f"HOTEL :{hotel['hotel']}, {hotel['price_per_night']}/night , {hotel['rating']}\n"
        f"TOTAL COST: {total_cost}"
    )

In [16]:
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
import os 
from dotenv import load_dotenv
load_dotenv()

llm = ChatGroq(
    model="llama-3.1-8b-instant",   
    api_key=os.getenv("GROQ_API_KEY"),
    temperature=0
)

llm_with_tools = llm.bind_tools(
    [search_flights , search_hotels , calculate_total_cost , format_flight_summary] 
)

In [17]:
thought_prompt = ChatPromptTemplate.from_messages([
    ("system",
     "You are a planning assistant. Given a travel request, generate 3 distinct, viable solution strategies. "
     "Each strategy should describe a high-level plan (not tool calls), including steps and priorities.\n\n"
     "Return the strategies as a numbered list.\n"
     "Do not execute tools.\n"
     "Do not include prices.\n"
    ),
    ("user", "{question}")
])

In [18]:
evaluation_prompt = ChatPromptTemplate.from_messages([
    ("system",
     "You are an expert evaluator. Given multiple solution strategies, evaluate each based on:\n"
     "- Feasibility\n"
     "- Cost-efficiency\n"
     "- Alignment with user constraints\n"
     "- Completeness\n\n"
     "Score each strategy from 1 to 10 and explain briefly.\n"
     "Then clearly state which strategy is best and why.\n"
    ),
    ("user", "{strategies}")
])


In [28]:
execution_prompt = ChatPromptTemplate.from_messages([
    ("system",
     "You are an intelligent travel assistant executing a selected plan.\n\n"
     "Follow the plan exactly.\n"
     "Use tools to fetch real data.\n"
     "Do not assume prices or schedules.\n"
     "Once all information is gathered, provide a structured final answer and stop.\n"
     "Do not reveal internal reasoning.\n"
    ),
    ("user", "{question}\n\nSelected Plan:\n{plan}")
])


- Everytime you invoke pass a list of msgs and

In [86]:
def tot_agent(question:str):
    strategies = llm_with_tools.invoke(thought_prompt.format_messages(question = question ))
    strategies_text = strategies.content
    print(f"Strategies:\n" , strategies_text)
    print(strategies.type)

    eval=llm_with_tools.invoke(evaluation_prompt.format_messages(strategies = strategies_text) )
    eval_text = eval.content 
    print(f"Evaluation:\n" , eval_text)
    print(eval.type)

    if "best strategy" in eval_text.lower():
        best_plan = eval_text.split("Best strategy:")[-1].strip()
    else:
        best_plan = strategies_text.split("\n")[0]

    print(f"Best_plan:\n" , best_plan)
    
    messages = execution_prompt.format_messages(question=question, plan=best_plan)
    assert isinstance(messages, list)
    print(f'-------------\n',messages)


    while True:
        response = llm_with_tools.invoke(messages)
        messages.append(response)

        if response.tool_calls:
            for tool_call in response.tool_calls:
                tool_name = tool_call["name"]
                args = tool_call["args"]
                
                tool_fn = {
                    "search_flights": search_flights,
                    "search_hotels": search_hotels,
                    "calculate_total_cost": calculate_total_cost,
                    "format_flight_summary": format_flight_summary
                }[tool_name]
                
                result = tool_fn.invoke(args)
                
                messages.append({
                    "role": "tool",
                    "tool_call_id": tool_call["id"],
                    "content": result
                })
                
        else:
            
            return response.content



In [87]:
result = tot_agent("I’m traveling from Bangalore to Delhi for 3 days. Find the cheapest flight, suggest a hotel within ₹4,000/night, and estimate total trip cost.")
print(result)


Strategies:
 1. Strategy: Budget-Friendly Option
- Step 1: Search for the cheapest flights from Bangalore to Delhi.
- Step 2: Book the cheapest flight found.
- Step 3: Search for a hotel in Delhi within the budget of ₹4,000/night.
- Step 4: Book the hotel that fits the budget.
- Step 5: Estimate the total trip cost by adding the flight and hotel costs.

2. Strategy: Mid-Range Option
- Step 1: Search for flights from Bangalore to Delhi with a balance between price and convenience.
- Step 2: Book a flight that meets the requirements.
- Step 3: Search for a hotel in Delhi within the budget of ₹4,000/night and with good reviews.
- Step 4: Book the hotel that fits the budget and meets the review requirements.
- Step 5: Estimate the total trip cost by adding the flight and hotel costs.

3. Strategy: Luxury Option
- Step 1: Search for flights from Bangalore to Delhi with a focus on luxury airlines.
- Step 2: Book a flight that meets the luxury requirements.
- Step 3: Search for a hotel in Del

In [88]:
messages

NameError: name 'messages' is not defined